1

In [1]:
# Cell 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set the dataset path
dataset_path = '/content/drive/MyDrive/country211'


Mounted at /content/drive


In [ ]:
# Cell 1: Mount Google Drive

# Set the dataset path
dataset_path = '/content/drive/MyDrive/country211'


2

3

In [4]:
# Cell 3: Verify Directory Structure
import os

# List the contents of the dataset directory
print("Contents of dataset directory:", os.listdir(dataset_path))

# List the contents of the train, val, and test directories
print("Contents of train directory:", os.listdir(f'{dataset_path}/train'))
print("Contents of val directory:", os.listdir(f'{dataset_path}/valid'))
print("Contents of test directory:", os.listdir(f'{dataset_path}/test'))

# Check for a few subdirectories and their contents
print("Contents of a sample train subdirectory:", os.listdir(f'{dataset_path}/train/AD'))
print("Contents of a sample val subdirectory:", os.listdir(f'{dataset_path}/valid/AD'))
print("Contents of a sample test subdirectory:", os.listdir(f'{dataset_path}/test/AD'))


Contents of dataset directory: ['train', 'test', 'valid']
Contents of train directory: ['VI', 'VN', 'YE', 'WS', 'XK', 'VU', 'ZM', 'ZA', 'ZW', 'UA', 'VG', 'UG', 'TZ', 'UY', 'UZ', 'VA', 'US', 'VE', 'TW', 'TG', 'TM', 'TJ', 'TL', 'TH', 'TN', 'TT', 'SZ', 'TR', 'TO', 'SX', 'SK', 'SJ', 'SY', 'SV', 'SO', 'SS', 'SL', 'SM', 'SN', 'RW', 'SG', 'RU', 'SA', 'SD', 'SC', 'SH', 'SI', 'SE', 'SB', 'PR', 'PS', 'PW', 'PL', 'RS', 'RO', 'PY', 'PT', 'QA', 'RE', 'PA', 'NP', 'PK', 'OM', 'PE', 'PF', 'PH', 'NZ', 'PG', 'NO', 'NA', 'MY', 'MX', 'NC', 'NI', 'NG', 'MV', 'NL', 'MW', 'MZ', 'ML', 'MK', 'MO', 'MQ', 'MT', 'MR', 'MM', 'MN', 'MG', 'MU', 'LT', 'LR', 'LV', 'MD', 'MC', 'LY', 'MA', 'LU', 'MF', 'ME', 'KZ', 'LC', 'LB', 'KR', 'KY', 'KW', 'LK', 'LI', 'KP', 'LA', 'JM', 'KG', 'KH', 'KN', 'JO', 'KE', 'JP', 'IS', 'JE', 'IT', 'IR', 'IN', 'IQ', 'IE', 'IL', 'HU', 'HR', 'IM', 'ID', 'HT', 'GL', 'GS', 'HN', 'HK', 'GP', 'GT', 'GY', 'GR', 'GU', 'GM', 'GF', 'GB', 'GI', 'GH', 'GA', 'GG', 'GD', 'FO', 'GE', 'FR', 'EC', 'EG', 'EE', 

4

In [6]:
# Cell 4: Data Loading and Preprocessing
# Import necessary libraries
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transformations for the training, validation, and test sets
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the datasets with ImageFolder
train_dataset = datasets.ImageFolder(root=f'{dataset_path}/train', transform=train_transforms)
val_dataset = datasets.ImageFolder(root=f'{dataset_path}/valid', transform=val_transforms)
test_dataset = datasets.ImageFolder(root=f'{dataset_path}/test', transform=val_transforms)

# Define the DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# Print dataset sizes
print(f'Training dataset size: {len(train_dataset)}')
print(f'Validation dataset size: {len(val_dataset)}')
print(f'Test dataset size: {len(test_dataset)}')


Training dataset size: 31671
Validation dataset size: 10550
Test dataset size: 21120


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


5

In [7]:
# Cell 5: Model Definition
# Import necessary libraries
import torch.nn as nn
import torch.nn.functional as F

# Define the CNN model
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 56 * 56, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 56 * 56)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Example usage (replace 'num_classes' with the actual number of classes in your dataset)
num_classes = len(train_loader.dataset.classes)
model = CNN(num_classes)
print(model)


CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=200704, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=211, bias=True)
)


6

In [ ]:
# Cell 6: Model Training
# Import necessary libraries
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the model, loss function, and optimizer
num_classes = len(train_loader.dataset.classes)
model = CNN(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Training function
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch {epoch}/{num_epochs - 1}, Loss: {epoch_loss:.4f}')

        scheduler.step()

    return model

model = train_model(model, criterion, optimizer, scheduler)


7

In [ ]:
# Cell 7: Model Evaluation
# Import necessary libraries
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def evaluate_model(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum
